In [1]:
!pip -q install scipy
!pip -q install numpy
!pip -q install tqdm
!pip -q install matplotlib
!pip -q install pandas

In [2]:
import pandas as pd

from utils import *

np.set_printoptions(precision=3)

In [3]:
path_log = './data/log.tsv'
path_topic = './data/topic.tsv'
path_sub_topic = './data/sub_topic.tsv'

In [4]:
def get_topics():
    topics = {}
    with open(path_topic) as f:
        for line in f.readlines()[1:]:
            items = line.split('\t')
            topic_id = int(items[0].strip())
            topic = items[1].strip().lower()
            topics[topic_id] = topic
    return topics


topics = get_topics()

print(topics)

{1: 'harvard university', 2: 'black death', 3: 'intergovernmental panel on climate change', 4: 'private schools', 5: 'geology', 6: 'economic inequality', 7: 'immune system', 8: 'oxygen', 9: 'normans', 10: 'amazon rainforest', 11: 'european union law'}


In [5]:
def get_users():
    users = defaultdict(int)
    to_filter = set()

    with open(path_log) as f:
        for line in f.readlines()[1:]:
            user = line.split('\t')[3].strip()
            if user.startswith('A'):
                users[user] += 1
            else:
                to_filter.add(user)

    for user in list(users.keys()):
        if users[user] < 10:
            del users[user]
            to_filter.add(user)

    return list(users.keys()), list(to_filter)


users, users_to_filter = get_users()

users_to_filter.append('AOWLCXNXCSQAC')
users_to_filter.append('A3P7QP6K56ED2Z')
users_to_filter.append('A2MOC4PTJYY15B')
users_to_filter.append('A1UZU88V9BMFT7')
users_to_filter.append('AWLT46A9EWTK8')
users_to_filter.append('A3GITKS8VABUNB')
users_to_filter.append('A6KB4VHCZYTY2')

with open(path_log) as f:
    with open('./data/log.1.tsv', 'w') as o:
        for n, line in enumerate(f.readlines()):
            if n == 0:
                o.write(line)
            else:
                user = line.split('\t')[3].strip()
                if user not in users_to_filter:
                    o.write(line)

path_log = './data/log.1.tsv'

In [6]:
users_topics_n = {}

for user in users:
    users_topics_n[user] = {}
    for topic in topics:
        with open(path_log) as f:
            task_completed = False
            queries = set()
            r_queries = []
            selected_lines = []
            for line in f.readlines():
                items = line.split('\t')
                if items[3].strip() == user and int(items[4]) == topic:
                    selected_lines.append(line)
                    if "queries" in items[2]:
                        queries.add(items[2].lower())
                        r_queries.append(items[2].lower())
                    if items[2] == 'task completed':
                        task_completed = True

            if len(queries) < 1 or len(queries) / len(r_queries) < 0.3 or not task_completed:
                selected_lines = []

            if len(selected_lines) > 7:
                users_topics_n[user][topic] = len(selected_lines) - 7
                # uncomment to manually check
                print(len(queries), len(r_queries))
                print(' '.join([line for line in selected_lines if "queries" in line]))

users_topics_n

1 1
2737	2020-02-19 16:15:13	user queries subtopic 72 with: what can be done to improve Intergovernmental Panel on Climate Change	AWJUGWPCUGKEG	3

2 2
669	2020-02-19 14:17:01	user queries subtopic 61 with: what is the immune system	AWJUGWPCUGKEG	7
 697	2020-02-19 14:17:31	user queries subtopic 61 with: what organs are in it and what are the organs for	AWJUGWPCUGKEG	7

1 1
79	2020-02-19 14:06:11	user queries subtopic 34 with: facts about normans	AWJUGWPCUGKEG	9

2 2
398	2020-02-19 14:11:51	user queries subtopic 51 with: what was the European Union Law	AWJUGWPCUGKEG	11
 446	2020-02-19 14:12:41	user queries subtopic 51 with: what country founded it	AWJUGWPCUGKEG	11

7 8
135	2020-02-19 14:07:21	user queries subtopic 1 with: Amazon rainforest	A2R89RXGEVG05I	10
 178	2020-02-19 14:08:20	user queries subtopic 2 with: Amazon rainforest location	A2R89RXGEVG05I	10
 212	2020-02-19 14:08:45	user queries subtopic 1 with: Amazon rainforest	A2R89RXGEVG05I	10
 261	2020-02-19 14:09:24	user queries subto

{'AWJUGWPCUGKEG': {3: 3, 7: 6, 9: 3, 11: 6},
 'AY6FIVEJJAWMK': {},
 'A2R89RXGEVG05I': {10: 24},
 'A1PTH9KTRO06EG': {1: 15,
  2: 15,
  3: 18,
  4: 18,
  5: 12,
  6: 12,
  7: 12,
  8: 12,
  9: 9,
  10: 9},
 'AYW62R027PUT1': {2: 21},
 'A2S64AUN7JI7ZS': {9: 12},
 'A1XKL37QKFITT1': {5: 39},
 'A2V66KLFVJTLKC': {2: 15},
 'A354IL6K61ZWHP': {9: 6},
 'A2EED3HLTA96CP': {10: 18},
 'A1WH32GYQOHIRQ': {10: 39},
 'A105ZUIAGMN8SW': {10: 24},
 'A1WCRV1WEWA63I': {6: 21},
 'A311A0TCGNTFKK': {10: 6},
 'A1AF25FCVKC87X': {6: 6},
 'A8C3WNWRBWUXO': {4: 33},
 'AERTM9Y12LBNF': {5: 6},
 'ARWN6TT8EWCFJ': {},
 'A36K4ELT4YH00L': {},
 'AOWLCXNXCSQAC': {},
 'A248QG4DPULP46': {10: 30},
 'A1TR615HGENX1M': {4: 15},
 'A3141EW4WZT4Q0': {4: 12},
 'A3PIUNG7VXQWTR': {7: 18},
 'A39GADIK8RLMVC': {2: 27, 6: 21},
 'A1OWHPMKE7YAGL': {},
 'A1GKD3NG1NNHRP': {10: 12},
 'A31XNLA9REVGXL': {9: 15},
 'A2M6O2Z15SHWDQ': {2: 15},
 'A2V4DP31BEVABF': {1: 18, 5: 33},
 'A2VL807897JLT9': {8: 42},
 'A2S5GPVIUV8RJI': {9: 8},
 'A3HN4CIWMVWYVF': {},

In [7]:
len(users_topics_n.keys())

220

In [8]:
with open(path_log) as f:
    with open('./data/log.2.tsv', 'w') as o:
        for n, line in enumerate(f.readlines()):
            if n == 0:
                o.write(line)
            else:
                items = line.split('\t')
                user = items[3].strip()
                topic = int(items[4].strip())
                if user in users_topics_n and topic in users_topics_n[user]:
                    o.write(line)

path_log = './data/log.2.tsv'

In [9]:
data = pd.read_csv("./rejections/Batch_3926170_batch_results.csv")

rejects = []
approves = []
for i_row, row in data.iterrows():
    worker_id = row['WorkerId']
    topic = row['Answer.topic']
    reject = True
    if topic.lower().strip() in topics.values():
        topic_id = None
        for key in topics:
            if topic.lower().strip() == topics[key]:
                topic_id = key
                break
        if worker_id in users_topics_n and topic_id in users_topics_n[worker_id]:
            reject = False

    if reject:
        rejects.append('X')
        approves.append('')
    else:
        rejects.append('')
        approves.append('X')

data['Approve'] = approves
data['Reject'] = rejects

data.to_csv('./rejections/Batch_3926170_batch_results.checked.csv')